In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
original_goodread_df = pd.read_csv('corrected_goodread_df.csv', error_bad_lines = False)
original_goodread_df.head()

,bookID,title,author,average_rating,isbn,isbn13,language_code,num_pages,audio_hrs,ratings_count,text_reviews_count,publication_date,publisher,series
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,439785960,9.780440e+12,eng,652,0.0,2095690,27591,2006-09-16,Scholastic Inc.,True
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,439358078,9.780439e+12,eng,870,0.0,2153167,29221,2004-09-01,Scholastic Inc.,True
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.780440e+12,eng,352,0.0,6333,244,2003-11-01,Scholastic,True
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,043965548X,9.780440e+12,eng,435,0.0,2339585,36325,2004-05-01,Scholastic Inc.,True
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling,4.78,439682584,9.780440e+12,eng,2690,0.0,41428,164,2004-09-13,Scholastic,True


In [3]:
goodread_df = original_goodread_df.copy()

In [4]:
goodread_df.dtypes

bookID                  int64
title                  object
author                 object
average_rating        float64
isbn                   object
isbn13                float64
language_code          object
num_pages               int64
audio_hrs             float64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
series                   bool
dtype: object

In [5]:
#create copy of df to fix datatypes
goodread_copy_df = goodread_df.copy()
goodread_copy_df.head()

goodread_copy_df.dtypes

#update isbn13 to object
goodread_copy_df['isbn13'] = goodread_copy_df['isbn13'].astype(str)

#update publication_date to datetime
goodread_copy_df['publication_date'] = pd.to_datetime(goodread_copy_df['publication_date'])
  
goodread_copy_df.dtypes

bookID                         int64
title                         object
author                        object
average_rating               float64
isbn                          object
isbn13                        object
language_code                 object
num_pages                      int64
audio_hrs                    float64
ratings_count                  int64
text_reviews_count             int64
publication_date      datetime64[ns]
publisher                     object
series                          bool
dtype: object

In [6]:
goodread_copy_df.sort_values(by='text_reviews_count', ascending=False)

,bookID,title,author,average_rating,isbn,isbn13,language_code,num_pages,audio_hrs,ratings_count,text_reviews_count,publication_date,publisher,series
9653,41865,Twilight (Twilight #1),Stephenie Meyer,3.59,316015849,9780316015844.0,eng,501,0.0,4597666,94265,2006-09-06,Little Brown and Company,True
4916,19063,The Book Thief,Markus Zusak,4.37,375831002,9780375831003.0,eng,552,0.0,1516367,86881,2006-03-14,Alfred A. Knopf,False
996,3636,The Giver (The Giver #1),Lois Lowry,4.13,385732554,9780385732550.0,eng,208,0.0,1585589,56604,2006-01-24,Ember,True
269,865,The Alchemist,Paulo Coelho,3.86,61122416,9780061122415.0,eng,197,0.0,1631221,55843,1993-05-01,HarperCollins,False
9991,43641,Water for Elephants,Sara Gruen,4.09,1565125606,9781565125605.0,eng,335,0.0,1260027,52759,2007-05-01,Algonquin Books,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6178,25060,Brightness Falls,Jay McInerney,3.74,747553718,9780747553717.0,eng,432,0.0,46,0,2001-08-06,Bloomsbury,False
6152,25002,Writing Degree Zero,Roland Barthes,3.88,809013843,9780809013845.0,eng,96,0.0,19,0,1977-05-01,Hill and Wang,False
6117,24894,Once Said Darlene,William Sleator,3.62,525364102,9780525364108.0,eng,55,0.0,8,0,1979-03-22,Dutton Books,False
6063,24749,American Writers Supplement VIII,Jay Parini,0.00,684806371,9780684806372.0,eng,400,0.0,0,0,2001-05-01,Gale Cengage,False


In [7]:
goodread_copy_df.sort_values(by='ratings_count', ascending=False)

,bookID,title,author,average_rating,isbn,isbn13,language_code,num_pages,audio_hrs,ratings_count,text_reviews_count,publication_date,publisher,series
9653,41865,Twilight (Twilight #1),Stephenie Meyer,3.59,316015849,9780316015844.0,eng,501,0.0,4597666,94265,2006-09-06,Little Brown and Company,True
1591,5907,The Hobbit or There and Back Again,J.R.R. Tolkien,4.27,618260307,9780618260300.0,eng,366,0.0,2530894,32871,2002-08-15,Houghton Mifflin,False
1363,5107,The Catcher in the Rye,J.D. Salinger,3.80,316769177,9780316769174.0,eng,277,0.0,2457092,43499,2001-01-30,Back Bay Books,False
291,960,Angels & Demons (Robert Langdon #1),Dan Brown,3.89,1416524797,9781416524793.0,eng,736,0.0,2418736,21303,2006-04-01,Pocket Books,True
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,043965548X,9780439655484.0,eng,435,0.0,2339585,36325,2004-05-01,Scholastic Inc.,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,12225,A Streetcar Named Desire (SparkNotes Literatur...,SparkNotes,3.29,1586634496,9781586634490.0,eng,96,0.0,0,0,2002-07-15,SparkNotes,False
7482,30941,Choo Choo (Carry Along Book & Cassette Favorites),Virginia Lee Burton,3.91,395511682,9780395511688.0,eng,48,0.0,0,0,1989-04-24,Houghton Mifflin,False
3275,12712,Brodie's notes on Aldous Huxley's brave new world,Graham Handley,0.00,333581296,9780333581292.0,eng,71,0.0,0,0,1992-08-20,Macmillan,False
7238,30080,Beckett Football Card Price Guide,Dan Hitt,0.00,1930692471,9781930692473.0,eng,830,0.0,0,0,2006-09-01,Beckett Media,False


In [8]:
goodread_copy_df.sort_values(by='average_rating', ascending=False)

,bookID,title,author,average_rating,isbn,isbn13,language_code,num_pages,audio_hrs,ratings_count,text_reviews_count,publication_date,publisher,series
9226,39829,His Princess Devotional: A Royal Encounter Wit...,Sheri Rose Shepherd,5.0,1590529626,9781590529621.0,eng,240,0.0,2,0,2007-10-16,Multnomah,False
4463,17224,The Diamond Color Meditation: Color Pathway to...,John Diamond,5.0,1890995525,9781890995522.0,eng,74,0.0,5,3,2006-02-01,Square One Publishers,False
8649,36492,Oliver Wendell Holmes in Paris: Medicine Theo...,William C. Dowling,5.0,1584655801,9781584655800.0,eng,179,0.0,1,1,2007-02-28,University Press of New England,False
5267,20954,Winchester Shotguns,Dennis Adler,5.0,785821082,9780785821083.0,eng,372,0.0,2,0,2008-05-15,Chartwell Books,False
5268,20957,Colossians and Philemon: A Critical and Exeget...,R. McL. Wilson,5.0,567044718,9780567044716.0,eng,512,0.0,1,0,2005-12-07,T&T Clark Int'l,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,25841,The Road To War 1933 39,Andrew Hunt,0.0,340774770,9780340774779.0,eng,128,0.0,0,0,2000-04-01,Hodder & Stoughton Educational Division,False
7021,29091,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,0.0,9027712050,9789027712059.0,eng,210,0.0,0,0,1981-05-31,Springer,False
3275,12712,Brodie's notes on Aldous Huxley's brave new world,Graham Handley,0.0,333581296,9780333581292.0,eng,71,0.0,0,0,1992-08-20,Macmillan,False
6361,25903,I'll Be Home Before Midnight and I Won't Get P...,Anthony E. Wolf,0.0,394755669,9780394755663.0,eng,288,0.0,0,0,1988-05-12,Knopf Doubleday Publishing Group,False


In [9]:
goodread_copy_df.sort_values(by='publication_date', ascending=False)

,bookID,title,author,average_rating,isbn,isbn13,language_code,num_pages,audio_hrs,ratings_count,text_reviews_count,publication_date,publisher,series
470,1571,Clouds,Aristophanes,3.75,199120099,9780199120093.0,eng,254,0.0,13,2,2069-12-15,Oxford University Press USA,False
5188,20323,The Penguin Companion to European Literature,Anthony Thorlby,4.50,140510354,9780140510355.0,eng,907,0.0,1,0,2069-05-30,Penguin Books,False
437,1516,Aeschylus I: Oresteia (Agamemnon The Libation...,Aeschylus,4.02,226307786,9780226307787.0,eng,171,0.0,712,53,2069-05-15,University of Chicago Press,False
451,1536,Sophocles II: Ajax/Women of Trachis/Electra/Ph...,Sophocles,4.14,226307867,9780226307862.0,eng,254,0.0,5032,36,2069-05-15,University of Chicago Press,False
3257,12645,Aspects of the Theory of Syntax,Noam Chomsky,3.96,262530074,9780262530071.0,eng,261,0.0,296,10,2069-03-15,MIT Press (Cambridge MA),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,1586,Lysistrata,Aristophanes,3.85,451616227,9780451616227.0,eng,98,0.0,19,2,1970-02-01,Signet,False
6027,24601,Paris Spleen,Charles Baudelaire,4.30,811200078,9780811200073.0,eng,118,0.0,7792,176,1970-01-17,New Directions,False
123,253,The Air-Conditioned Nightmare,Henry Miller,3.83,811201066,9780811201063.0,eng,292,0.0,2699,142,1970-01-17,New Directions,False
4665,18132,The Voyage of the “Dawn Treader” (The Chronicl...,C.S. Lewis,4.09,20442602,9780020442608.0,eng,216,0.0,2977,250,1970-01-01,Macmillan Publishing Company/Collier Books,True


### Questions to look into:
#### What gender of the author breakdown in the top 20 highest-rated books vs the bottom 20 lowest-rated books?
#### What is the skin color breakdown in the top 20 highest-rated books vs the bottom 20 lowest-rated books?
#### What is the fiction/non-fiction breakdown in the top 20 highest-rated books vs the bottom 20 lowest-rated books?
#### What is the series/not-a-series breakdown in the top 20 highest-rated books vs the bottom 20 lowest-rated books?
#### What is the relationship between book length and being at either extreme in the highest- or lowest-rated books?

### Other questions to look into:
#### What is the gender breakdown in the top 20 most-rated books vs the bottom 20 least-rated books?
#### What is the skin color breakdown in the top 20 most-rated books vs the bottom 20 least-rated books?
#### What is the fiction/non-fiction breakdown in the top 20 most-rated books vs the bottom 20 least-rated books?
#### What is the series/not-a-series breakdown in the top 20 most-rated books vs the bottom 20 least-rated books?
#### What is the relationship between book length and being at either extreme in the most- or least-rated books?

#### What gender is in the top 20 most-reviewed books vs the bottom 20 least-reviewed books?
#### What is the skin color breakdown in the top 20 most-reviewed books vs the bottom 20 least-reviewed books?
#### What is the fiction/non-fiction breakdown in the top 20 most-reviewed books vs the bottom 20 least-reviewed books?
#### What is the series/not-a-series breakdown in the top 20 most-reviewed books vs the bottom 20 least-reviewed books?
#### What is the relationship between book length and being at either extreme in the most- or least-rated books?